In [1]:
import pandas as pd
import numpy as np
import statistics
import csv
import numbers

Будущих бухгалтеров отталкивает сложность современных электронных таблиц. Разработайте для них программу с минимальным функционалом: хранение числовых и текстовых данных в таблице с изменяемым размером.
1. Ячейка электронной таблицы: числовое или текстовое значение, может быть пустой
2. Ячейка с формулой: диапазон ячеек (адреса первой и последней ячейки) и операция над диапазоном (сумма, произведение, среднее значение), метод вывода результата операции (или ошибки, если расчёт невозможен).
3. Таблица: хранение ячеек и вычисления операций над ними.


In [2]:
class Cell:
    def __init__(self, value=None):
        self.value = value
        self.type = type(self.value)
        
    def change(self, value):
        self.value = value
        self.type = type(self.value)
    


In [3]:
class CellOperation:
    def __init__(self, cells):
            self.cells = cells
            self.c_values = [c.value for c in cells]
            self.value = None
    def add(self):
        try:
            res = sum(self.c_values)
            self.value = res
            print(res)
        except:
            print('Расчет невозможен')
    def mult(self):
        try:
            res = np.prod(self.c_values)
            self.value = res
            print(res)
        except:
            print('Расчет невозможен')
    def mean(self):
        try:
            res = np.mean(self.c_values)
            self.value = res
            print(res)
        except:
            print('Расчет невозможен')

In [4]:
class SimpleTable:
    def __init__(self, file, encoding = 'utf-8-sig'):
        self.table = []
        with open(file, encoding=encoding, newline='') as File:  
            reader = csv.reader(File, delimiter=';')
            for row in reader:
                for i in range(0,len(row)):
                    if row[i] == '':
                        row[i] = Cell()
                    else:
                        try:
                            row[i] = Cell(value = int(row[i]))
                        except:
                            row[i] = Cell(value = row[i])
                self.table.append(row)
    
    def print_table(self):
        mapper = lambda x: x+1
        table = [[c.value for c in row] for row in self.table]
        df = pd.DataFrame(data=table)
        df.rename(mapper=mapper, axis=1, inplace=True)
        df.rename(mapper=mapper, axis=0, inplace=True)
        print(df)
        
    def cell(self, row, col):
        """
        Возвращает значения в ячейке
        rows-1 и col-1 -- чтобы адрес ячейки можно было задавать не 
        в питоновском формате, а в человеческом (значения начинаются с 1)
        """
        return self.table[row-1][col-1]
        
        
    def cell_change(self, row, col, value):
        self.table[row-1][col-1].change(value)
        
    def add_row(self, row_pos=-9999, row_values=[]):
        if row_pos == -9999:
            row_pos = len(self.table) + 1
        if row_pos > (len(self.table)+1) or row_pos < 1 or isinstance(row_pos, int) == False:
            return 'Row index out of range'
        if row_values == []:
            N = len(self.table[0])
            row_values = np.array([None] * N)
        if len(row_values) != len(self.table[0]):
            return 'Недопустимая длина строки'
        row_pos = row_pos - 1
        row = [Cell(value=i) for i in row_values]
        self.table = np.insert(self.table, row_pos, row, axis=0)
        self.print_table()
    
    def add_col(self, col_pos=-9999, col_values=[]):
        if col_pos == -9999:
            col_pos = len(self.table[0]) + 1
        if col_pos > (len(self.table[0])+1) or col_pos < 1 or isinstance(col_pos, int) == False:
            return 'Column index out of range'
        if col_values == []:
            N = len(self.table)
            col_values = np.array([None] * N)
        if len(col_values) != len(self.table):
            return 'Недопустимая длина столбца'
        col_pos = col_pos - 1
        col = [Cell(value=i) for i in col_values]
        self.table = np.insert(self.table, col_pos, col, axis=1)
        self.print_table()
    
    def delete_row(self, row_pos):
        if row_pos > len(self.table) or row_pos < 1 or isinstance(row_pos, int) == False:
            return 'Row index out of range'
        else:
            row_pos = row_pos - 1
            self.table = np.delete(self.table, row_pos, axis=0)
            self.print_table()
        
    def delete_col(self, col_pos):
        if col_pos > len(self.table[0]) or col_pos < 1 or isinstance(col_pos, int) == False:
            return 'Column index out of range'
        else:
            col_pos = col_pos - 1
            self.table = np.delete(self.table, col_pos, axis=1)
            self.print_table()
    
        
    def set_range(self, row1, col1, row2, col2):
        """
        Возвращает массив значений из заданного диапазона
        """
        cells = []
        if row1 == row2 & col1 == col2:
            cells = self.table[row1-1][col1-1]
            print('В диапазоне только 1 ячейка')
        elif row1 == row2:
            a = min(col1,col2)
            b = max(col1,col2)
            for i in range (a-1,b):
                cells.append(self.table[row1-1][i])
        elif col1 == col2:
            a = min(row1,row2)
            b = max(row1,row2)
            for i in range (a-1,b):
                cells.append(self.table[i][col1-1])
        else:
            print('Неверно указан диапазон')
        return cells
    
    def range_operation(self, cell_row, cell_col, row1, col1, row2, col2):
        self.table[cell_row-1][cell_col-1] = CellOperation(cells = self.set_range(row1, col1, row2, col2))
        


Таблица из файла

In [7]:
tb = SimpleTable('test.csv')
tb.print_table()

   1     2  3   4
1  a     b  3   d
2  1  None  2   3
3  2   sff  3  sd
4  1     2  3   4


Добавляем ряд (2 ряд, значения [2,'q',3,'w'])

In [8]:
tb.add_row(2,[2,'q',3,'w'])

   1     2  3   4
1  a     b  3   d
2  2     q  3   w
3  1  None  2   3
4  2   sff  3  sd
5  1     2  3   4


In [9]:
tb.add_row(10,[2,'q',3,'w'])

'Row index out of range'

Добавляем столбец (без уточнения добавляется последний столбец с пустыми значениями)

In [10]:
tb.add_col()

   1     2  3   4     5
1  a     b  3   d  None
2  2     q  3   w  None
3  1  None  2   3  None
4  2   sff  3  sd  None
5  1     2  3   4  None


Удалить столбец

In [11]:
tb.delete_col(5)

   1     2  3   4
1  a     b  3   d
2  2     q  3   w
3  1  None  2   3
4  2   sff  3  sd
5  1     2  3   4


В [1,1] создаем ячейку с формулой (операция над ячейками [1,3]-[4,3])

In [12]:
tb.range_operation(1,1,1,3,4,3)

In [14]:
tb.cell(1,1).c_values

[3, 3, 2, 3]

In [15]:
tb.cell(1,1).mult()

54


In [16]:
tb.print_table()

    1     2  3   4
1  54     b  3   d
2   2     q  3   w
3   1  None  2   3
4   2   sff  3  sd
5   1     2  3   4


In [17]:
tb.cell(1,1).add()
tb.print_table()

11
    1     2  3   4
1  11     b  3   d
2   2     q  3   w
3   1  None  2   3
4   2   sff  3  sd
5   1     2  3   4
